<a href="https://colab.research.google.com/github/CatalystM47/Deep_Learning/blob/main/randominvert(p%3D0.5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Augmentation

In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv("sign_mnist_train.csv")
valid_df = pd.read_csv("sign_mnist_valid.csv")

In [2]:
y_train = train_df["label"]
y_valid = valid_df["label"]

In [3]:
del train_df["label"]
del valid_df["label"]

x_train = train_df.values
x_valid = valid_df.values

In [4]:
from PIL import Image, ImageOps

image = x_train[0,:].reshape(28,28)
img_pil = Image.fromarray(np.uint8(image))

In [5]:
class Invert(object):
    
    def invert(self):
        if img.mode == 'LA':
            l, a = img.split()
            l = ImageOps.invert(l)
            inv = Image.merge('LA', (l, a))
        else:
            inv = ImageOps.invert(img)
        return inv

    def __call__(self, img):
        return self.invert(img)

In [6]:
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.Resize(32),
     transforms.RandomCrop(28),
     transforms.RandomInvert(p=0.5)]
)

In [7]:
img_pil

In [25]:
transform(img_pil)

### 1. Data preparation

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader

In [10]:
class ASL_Dataset(Dataset):
    def __init__(self, path, transform=None):
        super(ASL_Dataset, self).__init__()

        train_df = pd.read_csv(path)
        y_train = train_df["label"]
        del train_df["label"]
        x_train = train_df.values

        self.x_train = x_train   # Nx784
        self.y_train = y_train
        self.transform = transform

    def __getitem__(self, index):
        x_train_sample = np.uint8( self.x_train[index].reshape(28,28) )

        if self.transform:
            x_train_pil = Image.fromarray(x_train_sample)
            x_train_np = np.array(self.transform(x_train_pil))
            x_train_tensor = torch.tensor( x_train_np[np.newaxis,:,:] ).float() # 1x28x28
        else:
            x_train_tensor = torch.tensor( x_train_sample[np.newaxis,:,:] ).float()

        y_train_tensor = torch.tensor( self.y_train[index] )

        return x_train_tensor, y_train_tensor

    def __len__(self):
        return self.x_train.shape[0]

In [11]:
training_data = ASL_Dataset("sign_mnist_train.csv", transform=transform)
validation_data = ASL_Dataset("sign_mnist_valid.csv")

In [12]:
train_loader = DataLoader(dataset=training_data, batch_size=1000, shuffle=True)
validation_loader = DataLoader(dataset=validation_data, batch_size=1000, shuffle=False)

### 2. Constructing a convolutional neural network

In [13]:
import torch.nn as nn

In [14]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(32*7*7, 24)

    def forward(self, x):  ## 1x28x28
        x = self.conv1(x)   ## 16x28x28
        x = self.relu(x)
        x = self.pool(x)    ## 16x14x14

        x = self.conv2(x)   ## 32x14x14
        x = self.relu(x)
        x = self.pool(x)    ## 32x7x7

        x = self.conv3(x)   ## 32x7x7
        x = self.relu(x)

        x = x.view(-1, 32*7*7)
        x = self.fc(x)

        return x

In [15]:
convnet = ConvNet()

### 3. Loss function and Optimization method

In [16]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnet.parameters(), lr=0.001)

### 4. Training of a neural network 

In [17]:
for epoch in range(20):
    loss_val = 0
    for itr, data in enumerate(train_loader):
        optimizer.zero_grad()

        inputs, labels = data

        pred = convnet(inputs)
        loss = loss_function(pred, labels)

        loss.backward()
        optimizer.step()

        loss_val += loss.item()

    print("Epoch:", epoch+1, "  , Loss:", loss_val)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 1   , Loss: 109.1374397277832
Epoch: 2   , Loss: 61.61223554611206
Epoch: 3   , Loss: 33.02785241603851
Epoch: 4   , Loss: 20.248279988765717
Epoch: 5   , Loss: 15.248484790325165
Epoch: 6   , Loss: 10.97213500738144
Epoch: 7   , Loss: 8.771793156862259
Epoch: 8   , Loss: 7.167286932468414
Epoch: 9   , Loss: 5.668005526065826
Epoch: 10   , Loss: 5.056821718811989
Epoch: 11   , Loss: 4.701211541891098
Epoch: 12   , Loss: 3.7529379054903984
Epoch: 13   , Loss: 3.172401651740074
Epoch: 14   , Loss: 2.7805152013897896
Epoch: 15   , Loss: 2.400572396814823
Epoch: 16   , Loss: 2.3153011687099934
Epoch: 17   , Loss: 2.1810060553252697
Epoch: 18   , Loss: 1.9870590642094612
Epoch: 19   , Loss: 1.6195061802864075
Epoch: 20   , Loss: 1.5681490823626518


### 5. Prediction and evaludation for a test set

In [18]:
pred_list = []
label_list = []
for itr, data in enumerate(validation_loader):
    inputs, labels = data
    pred_test = convnet(inputs)
    pred_category = torch.argmax(pred_test, dim=1)

    pred_list = pred_list + list(pred_category)    
    label_list = label_list + list(labels)

accu = np.mean( np.array(pred_list)==np.array(label_list) )
print("Test accuracy:", accu)

Test accuracy: 0.8877579475738985
